# Analisi iscrizioni

Questo *notebook* consente di ottenere la "storia" della partecipazione all'esame degli studenti iscritti ad un determinato apello a partire dai verbali estratti tramite le funzione *Archivio verbali ed esiti finali* e *Lista appelli ed Iscritti* dell'applicazione [B.O. Docent](https://work.unimi.it/boDocenti/); più in generale consente di svolgere le analisi a partire da un qualunque elenco di verbali d'esame che registrino, per ciascuna prova, la *data* in cui è avvenuta, il numero di *matricola* dello studente (o un qualunque identificatore unico del medesimo) e l'*esito* della prova, nonché da un elenco di iscrizioni che registrino, per ciascuna iscrizione, il numero di *matricola*, *cognome* e *nome* dello studente

Per sapere come installare il software necessario ad eseguirlo seguite le istruzioni contenute nel `README.md` del repository che contiene il notebook.

In [ ]:
# importazione delle librerie necessarie

import pandas as pd
from ipywidgets import widgets

## Lettura e preparazione dei dati

In questa sezione vengono caricati e "normalizzati" i dati che si assume siano stati ottenuti dall'esportazione sopra descritta e salvanti in un file il cui nome è memorizzato nella variabile `file_verbali`.

Eseguita l'ultima cella di questa sezione i dati saranno contenuti nella variabile `verbali` che corrisponderà ad un *data-frame* di Pandas contenente le colonne: `Inesegnamento`, `Appello`, `Matricola` ed `Esito` ed ordinato per data dell'appello (l'ordine è rilevante per le aggregazioni delle prossime sezioni). Il contenuto della colonna `Eisto` è una *stringa* che codifica l'esito della prova come segue:

* `RI` corrisponde ad un ritiro,
* `RE` indica che la prova non è stata superata (ossia che l'allievo è stato respinto),
* un stringa contenente un numero intero corrisponde ad un esito verbalizzato (la stringa `33` corrisponde al voto 30 e lode),
* una stringa contenente un numero intero prefissato dal simbolo `~` corrisponde ad un esito rifiutato. 

In [ ]:
file_verbali = 'verbali-esami.xls'

In [ ]:
def vse2e(voto, stato_esito):
    if stato_esito == 'Rifiutato': return '~' + voto
    if voto == 'Respinto': return 'RE'
    if voto == 'Ritirato': return 'RI'
    if voto == '30 e lode': return '33'
    return voto

verbali = pd.read_excel(file_verbali).apply(lambda _: pd.Series({
    'Insegnamento': _['Descrizione insegnamento'],
    'Appello': _['Data appello'],
    'Matricola': _['Matricola'], 
    'Esito': vse2e(_['Voto'], _['Stato Esito'])
}), axis = 1).sort_values('Appello')

In [ ]:
# vengono riportate alcune linee a scopo di esempio

verbali.head()

Similmente, vengono caricati e "normalizzati" i dati che si assume siano stati ottenuti dall'esportazione sopra descritta e salvanti in un file il cui nome è memorizzato nella variabile `file_iscrizioni`.

Eseguita l'ultima cella di questa sezione i dati saranno contenuti nella variabile `iscrizioni` che corrisponderà ad un *data-frame* di Pandas contenente le colonne: `Cognome`, `Nome`, indiizzato per `Matricola` 

In [ ]:
file_iscrizioni = 'iscrizioni.xls'

In [ ]:
iscrizioni = pd.read_excel(file_iscrizioni, dtype = {'Matricola': object})[['Matricola', 'Cognome', 'Nome']].set_index('Matricola')

In [ ]:
# vengono riportate alcune linee a scopo di esempio

iscrizioni.head()

# La storia di ciascun iscritto

Per "storia" di un iscritti si intende l'eventuale sequenza di esiti conseguiti in precedenti appelli del medesimo insegnamento, il cui nome è dato dalla variabile `nome_insegnamento`; la valutazione delle prossime celle produce, per ciascun numero di matricola presente nell'elenco di iscrizioni, una stringa ottenuta giustapponendo gli esiti precedenti, oppure la stringa `NI` (che significa "nuovo iscritto", ossia matricola a cui non corrisponde alcun verbale).

In [ ]:
nome_insegnamento = 'PROGRAMMAZIONE II'

In [ ]:
storia = ( verbali[verbali.Insegnamento == nome_insegnamento]
            .merge(iscrizioni, on = 'Matricola', how = 'right')
            .fillna('NI').groupby('Matricola').Esito
            .agg(lambda _: ', '.join(_)) )
storia.name = 'Storia'

In [ ]:
# un controllo circa il fatto che il numero di "storie" corrisponda a quello degli iscritti

storia.shape[0] == iscrizioni.shape[0]

In [ ]:
# riconcilare storia e iscrizioni

storia_iscrizioni = iscrizioni.join(storia).sort_values('Matricola')

In [ ]:
# come tabella, colorando le righe dei nuovi iscritti

storia_iscrizioni.style.apply((lambda _: ['background-color: lightyellow']*3 if _.Storia == 'NI' else ['']*3), axis = 1)